<a href="https://colab.research.google.com/github/kaneziki442/gpt2-lora-qa/blob/main/xuexi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving qa_train_50.jsonl to qa_train_50.jsonl


In [ ]:
!pip install -q datasets

from datasets import load_dataset

dataset = load_dataset("json", data_files="qa_train_50.jsonl", split="train")
for example in dataset:
    print(example)


Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'Q: What is machine learning?\nA: Machine learning is a field of artificial intelligence that uses algorithms to learn patterns from data.'}
{'text': 'Q: What is the capital of France?\nA: The capital of France is Paris.'}
{'text': 'Q: Who is Elon Musk?\nA: Elon Musk is the CEO of Tesla and SpaceX.'}
{'text': 'Q: What is Python?\nA: Python is a high-level programming language known for its readability and flexibility.'}
{'text': 'Q: What is the tallest mountain in the world?\nA: Mount Everest is the tallest mountain in the world.'}
{'text': 'Q: What is machine learning?\nA: Machine learning is a field of artificial intelligence that uses algorithms to learn patterns from data.'}
{'text': 'Q: What is the capital of France?\nA: The capital of France is Paris.'}
{'text': 'Q: Who is Elon Musk?\nA: Elon Musk is the CEO of Tesla and SpaceX.'}
{'text': 'Q: What is Python?\nA: Python is a high-level programming language known for its readability and flexibility.'}
{'text': 'Q: What is

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# 加载预训练模型
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # 修复 padding 报错

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_name)

# 配置 LoRA
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# 加载 LoRA 到模型上
model = get_peft_model(model, lora_config)

print("✅ 模型和LoRA准备完毕")


✅ 模型和LoRA准备完毕


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# 用 tokenizer 编码训练数据
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",         # 自动补齐
        truncation=True,              # 截断太长的
        max_length=128,               # 设置最大长度
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # 只保留 input_ids 等字段
tokenized_dataset.set_format("torch")  # 转成 PyTorch tensor

print("✅ 数据编码完成，准备训练！")
tokenized_dataset[0]


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

✅ 数据编码完成，准备训练！


{'input_ids': tensor([   48,    25,  1867,   318,  4572,  4673,    30,   198,    32,    25,
         10850,  4673,   318,   257,  2214,   286, 11666,  4430,   326,  3544,
         16113,   284,  2193,  7572,   422,  1366,    13, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 训练参数配置
training_args = TrainingArguments(
    output_dir="./lora-gpt2-results",         # 模型保存路径
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    report_to="none",                         # 不用 wandb 等工具
)

# 数据整理器（语言建模不使用MLM）
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer 初始化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 启动训练！
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.412300
20,2.312100
30,2.237300
40,2.179300
50,2.165400
60,1.997300
70,1.987000
80,1.862600
90,1.824000
100,1.745900


TrainOutput(global_step=250, training_loss=1.7085214805603028, metrics={'train_runtime': 688.4601, 'train_samples_per_second': 0.726, 'train_steps_per_second': 0.363, 'total_flos': 32718127104000.0, 'train_loss': 1.7085214805603028, 'epoch': 10.0})

In [ ]:
# 推理输入
input_text = "Q: Who developed GPT-2?\nA:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# 推理生成
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id  # 防止pad相关报错
)

print("🧠 生成结果：")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


🧠 生成结果：
Q: Who developed GPT-2?
A: The famous author John W. Smith is a Professor of Science and History at the University College London (UCL) School for International Studies and Sociology. He has written about more than 200 languages over his career including English, French, Italian/Italian
